In [3]:
# 1. 라이브러리 임포트
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# 2. 하이퍼파라미터 설정
batch_size = 64
learning_rate = 0.001
epochs = 5

# 3. 데이터 로딩 (MNIST, 흑백 → 1채널)
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = torchvision.datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, transform=transform, download=True)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# 4. CNN 모델 정의
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv_layer = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, padding=1),  # (1,28,28) → (16,28,28)
            nn.ReLU(),
            nn.MaxPool2d(2, 2),                         # (16,28,28) → (16,14,14)

            nn.Conv2d(16, 32, kernel_size=3, padding=1), # (16,14,14) → (32,14,14)
            nn.ReLU(),
            nn.MaxPool2d(2, 2)                          # (32,14,14) → (32,7,7)
        )
        self.fc_layer = nn.Sequential(
            nn.Flatten(),                               # (32*7*7)
            nn.Linear(32*7*7, 128),
            nn.ReLU(),
            nn.Linear(128, 10)                          # 10개 숫자 분류
        )

    def forward(self, x):
        x = self.conv_layer(x)
        x = self.fc_layer(x)
        return x

model = CNN()

# 5. 손실 함수와 옵티마이저
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# 6. 학습 루프
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader):.4f}")

# 7. 테스트 정확도 평가
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")


Epoch 1/5, Loss: 0.2097
Epoch 2/5, Loss: 0.0569
Epoch 3/5, Loss: 0.0394
Epoch 4/5, Loss: 0.0290
Epoch 5/5, Loss: 0.0235
Test Accuracy: 99.00%
